In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install scikit-learn==1.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 55.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.3.0 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings('ignore')

cust_df = pd.read_csv("/content/drive/MyDrive/자료/멀티캠퍼스/자료모음/santander.csv", encoding='latin-1')
print('dataset shape:', cust_df.shape)
cust_df.head(3)

dataset shape: (76020, 371)


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.17,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.03,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.77,0


산탄데르 은행 고객 만족도조사 결과
피처 - 익명화, 컬럼명 익명화, 수치, 0인 데이터가 대다수 - 희소성
타겟 - TARGET

# 데이터 탐색

In [ ]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


100명중 4명이 불만족하는 데이터

In [ ]:
cust_df['TARGET'].value_counts() #0 만족, 1 불만족
unsatisfied_cnt = cust_df[cust_df['TARGET']==1].TARGET.count()
total_cnt = cust_df.TARGET.count()

print(f'불만족 비율은 : {unsatisfied_cnt/total_cnt:.2f}')

불만족 비율은 : 0.04


최소값, Q1, Q2, Q3 가 0인 피처가 많다.
최소값 -999999.000000 인 피처 var3 > Nan > 중위값/최빈값 2로 대체

In [ ]:
cust_df.describe()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
count,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,...,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,76020.000000,7.602000e+04,76020.000000
mean,75964.050723,-1523.199277,33.212865,86.208265,72.363067,119.529632,3.559130,6.472698,0.412946,0.567352,...,7.935824,1.365146,12.215580,8.784074,31.505324,1.858575,76.026165,56.614351,1.172358e+05,0.039569
std,43781.947379,39033.462364,12.956486,1614.757313,339.315831,546.266294,93.155749,153.737066,30.604864,36.513513,...,455.887218,113.959637,783.207399,538.439211,2013.125393,147.786584,4040.337842,2852.579397,1.826646e+05,0.194945
min,1.000000,-999999.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.163750e+03,0.000000
25%,38104.750000,2.000000,23.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.787061e+04,0.000000
50%,76043.000000,2.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.064092e+05,0.000000
75%,113748.750000,2.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.187563e+05,0.000000
max,151838.000000,238.000000,105.000000,210000.000000,12888.030000,21024.810000,8237.820000,11073.570000,6600.000000,6600.000000,...,50003.880000,20385.720000,138831.630000,91778.730000,438329.220000,24650.010000,681462.900000,397884.300000,2.203474e+07,1.000000


In [ ]:
cust_df.var3.value_counts()

,count
var3,
2,74165
8,138
-999999,116
9,110
3,108
...,...
40,1
57,1
112,1


## 피처 선택 - id 삭제, var3 은 최빈값 2로 대체

In [ ]:
cust_df.drop('ID', axis=1, inplace=True)
cust_df['var3'].replace(-999999, 2, inplace=True)

cust_df.head()

,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


## X, y 데이터 분할


In [ ]:
X_features = cust_df.iloc[:, :-1]
y_labels = cust_df.iloc[:, -1]
X_features.shape, y_labels.shape

((76020, 369), (76020,))

## 훈련/테스트 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=0)
#X_train.count()
print(f'학습 데이터 shape : {X_train.shape}, 테스트 데이터 shape : {X_test.shape}')

train_cnt = y_train.count()
test_cnt = y_test.count()
print(f'학습 데이터 값의 비율 : {y_train.value_counts()/train_cnt}')
print(f'테스트 데이터  값의 비율 : {y_test.value_counts()/test_cnt}')

학습 데이터 shape : (60816, 369), 테스트 데이터 shape : (15204, 369)
학습 데이터 값의 비율 : TARGET
0    0.960964
1    0.039036
Name: count, dtype: float64
테스트 데이터  값의 비율 : TARGET
0    0.9583
1    0.0417
Name: count, dtype: float64


# DT 예측기

# RF 예측기

# XGBoost 예측기 p271

## 훈련 데이터 > 훈련:검증 분할

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

## XGBoost model

In [ ]:
%%time

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimator=500, learning_rate=0.05, random_state=156, early_stopping_rounds=100, eval_metric='auc')
xgb_clf.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr), (X_val, y_val)] )
#xgb_clf.fit(X_tr, y_tr, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

[0]	validation_0-auc:0.83603	validation_1-auc:0.80978
[1]	validation_0-auc:0.83783	validation_1-auc:0.81126
[2]	validation_0-auc:0.83892	validation_1-auc:0.81192
[3]	validation_0-auc:0.84090	validation_1-auc:0.81349
[4]	validation_0-auc:0.84187	validation_1-auc:0.81356
[5]	validation_0-auc:0.84355	validation_1-auc:0.81422
[6]	validation_0-auc:0.84534	validation_1-auc:0.81550
[7]	validation_0-auc:0.84658	validation_1-auc:0.81623
[8]	validation_0-auc:0.84727	validation_1-auc:0.81669
[9]	validation_0-auc:0.84794	validation_1-auc:0.81668
[10]	validation_0-auc:0.84980	validation_1-auc:0.81828
[11]	validation_0-auc:0.85105	validation_1-auc:0.81965
[12]	validation_0-auc:0.85217	validation_1-auc:0.81999
[13]	validation_0-auc:0.85301	validation_1-auc:0.82049
[14]	validation_0-auc:0.85558	validation_1-auc:0.82210
[15]	validation_0-auc:0.85662	validation_1-auc:0.82288
[16]	validation_0-auc:0.85811	validation_1-auc:0.82380
[17]	validation_0-auc:0.85864	validation_1-auc:0.82365
[18]	validation_0-au

0.8413222688684454

# 하이퍼오피트 파라미터 최적화

In [ ]:
import xgboost
import hyperopt
import sklearn
xgboost.__version__, hyperopt.__version__, sklearn.__version__

('2.1.3', '0.2.7', '1.3.0')

In [ ]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from sklearn.base import BaseEstimator, ClassifierMixin
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from hyperopt  import hp

In [ ]:
# Sklearn 호환 XGBoost 클래스 정의
class SklearnCompatibleXGBClassifier(XGBClassifier, BaseEstimator, ClassifierMixin):
    def __sklearn_tags__(self):
        return {"non_deterministic": True}

In [ ]:
from hyperopt.pyll.base import scope
import numpy as np

# 하이퍼파라미터 검색공간
search_space = { 'n_estimators': scope.int(hp.quniform('n_estimators', 50, 300, 10)),
                'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)),
                'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
                'subsample':hp.uniform('subsample', 0.5, 1.0),
                'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
                }

In [ ]:
from hyperopt import fmin, tpe, Trials, STATUS_OK
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


def objective_func_xgb(params):

    model = SklearnCompatibleXGBClassifier(
        n_estimators=100, #params['n_estimators'],
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        #subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=42,
        eval_metric='logloss'
    )
    #k-fold 추가

    score_mean = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return {'loss': -1*score_mean , 'status': STATUS_OK}

In [ ]:
trials = Trials()

best_params = fmin(
    fn=objective_func_xgb,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

100%|██████████| 50/50 [31:33<00:00, 37.86s/trial, best loss: -0.9610793206484439]


In [ ]:
best_params

{'colsample_bytree': 0.7209254543566499,
 'learning_rate': 0.0417467908221523,
 'max_depth': 8.0,
 'n_estimators': 120.0,
 'subsample': 0.9225838193485726}

In [ ]:
best_model = XGBClassifier(n_estimators = int(best_params['n_estimators']),
                           max_depth = int(best_params['max_depth']),
                           learning_rate=best_params['learning_rate'],
                           subsample=best_params['subsample'] ,
                           colsample_bytree=best_params['colsample_bytree'],
                           random_state=42,
                           eval_metric='logloss'

                           )

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [ ]:
%%time
eval_list = [(X_tr, y_tr), (X_val, y_val)]


from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
best_model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score

[0]	validation_0-logloss:0.21391	validation_1-logloss:0.21483
[1]	validation_0-logloss:0.20938	validation_1-logloss:0.21068
[2]	validation_0-logloss:0.20518	validation_1-logloss:0.20677
[3]	validation_0-logloss:0.20123	validation_1-logloss:0.20315
[4]	validation_0-logloss:0.19743	validation_1-logloss:0.19966
[5]	validation_0-logloss:0.19421	validation_1-logloss:0.19668
[6]	validation_0-logloss:0.19120	validation_1-logloss:0.19391
[7]	validation_0-logloss:0.18778	validation_1-logloss:0.19087
[8]	validation_0-logloss:0.18445	validation_1-logloss:0.18792
[9]	validation_0-logloss:0.18139	validation_1-logloss:0.18516
[10]	validation_0-logloss:0.17873	validation_1-logloss:0.18279
[11]	validation_0-logloss:0.17638	validation_1-logloss:0.18067
[12]	validation_0-logloss:0.17372	validation_1-logloss:0.17826
[13]	validation_0-logloss:0.17108	validation_1-logloss:0.17598
[14]	validation_0-logloss:0.16854	validation_1-logloss:0.17376
[15]	validation_0-logloss:0.16626	validation_1-logloss:0.17173
[1

0.8417032751710983

In [ ]:
from sklearn.metrics import roc_auc_score
xgb_roc_score = roc_auc_score(y_test, best_model.predict_proba(X_test)[:,1])
xgb_roc_score

0.8453955558827286

# 최적화 파라미터 기준 모델 평가

n_estimator=500
learning_rate = 0.15
max_depth = 5


# LGBM 추가
%%time
<!--
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

xgb_clf = XGBClassifier(n_estimator=500, learning_rate=0.05, random_state=156)
xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric='auc', eval_set=[(X_tr, y_tr), (X_val, y_val)])
xgb_roc_score = roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:, 1])
xgb_roc_score -->